In [38]:
import getpass
import os
from credentials import *

In [39]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint= Azure_OpenAI_OB_Endpoint ,
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key= Azure_OpenAI_OB_Key,
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy",
    #streaming=True,
)

## 1 - Test

In [40]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. 
            Pour avoir du contexte , sache que ce texte est extrait du {code}"""),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")]
)

In [41]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

contenu_ancien = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
contenu_nouveau = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
llm_output = chain.invoke(
    {
        "code": "Code monétaire et financier",
        "old_version": contenu_ancien,
        "new_version": contenu_nouveau,
        "input": "I love programming.",
    }
)

In [42]:
print(llm_output)

L'analyse des modifications apportées à l'article du Code monétaire et financier révèle une suppression notable. En effet, la nouvelle version de l'article a éliminé toute mention des "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts". 

### Ancienne Version
L'ancienne version de l'article incluait explicitement les "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" parmi les entités qui doivent s'assurer que les intermédiaires en opérations de banque et en services de paiement sont immatriculés conformément à l'article L. 519-3-1. De plus, ces prestataires étaient également mentionnés dans le contexte des intermédiaires ressortissants d'un État membre de l'Union européenne ou d'un État partie à l'accord sur l'Espace économique européen exerçant en France en régime de libre prestation de services ou de libre établissement.

### Nouvelle Version
L

## 2 - Function

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def llm_legal_change_com(code, old_version, new_version): # "Code monétaire et financier"
    

    prompt = ChatPromptTemplate.from_messages([("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. Tu n'inventeras rien et tu te baseras uniquement sur les données fournies
            Pour avoir du contexte , sache que ce texte est extrait du {code}"""),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "code": code,
        "old_version": old_version,
        "new_version": new_version})
    
    return print(llm_output)

In [44]:
llm_legal_change_com("Code monétaire et financier", contenu_ancien, contenu_nouveau)

L'analyse des modifications apportées à l'article en question révèle une suppression spécifique dans la nouvelle version par rapport à l'ancienne. Plus précisément, la nouvelle version de l'article a omis la mention des "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" dans les deux paragraphes.

### Ancienne Version :
1. **Premier paragraphe** : Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recouren

## 3 - Data prep


In [46]:
import pandas as pd
AML5_modifs= pd.read_excel("/Users/oussa/Desktop/Github_perso/3 - Legal FR Tracker/data_output/Output_AML_5.xlsx")

In [48]:
# Filter on AML text
AML5_modifs =  AML5_modifs[AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-1544")| AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-115") ]

In [54]:
#check results
AML5_modifs['Titre Article Modificateur'].unique()

array(['Ordonnance n°2020-1544 du 9 décembre 2020 - art. 1',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 2',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 3',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 4',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 7',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 8',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 9',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 10',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 1',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 2',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 3',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 4',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 5',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 6',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 7',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 8',
       'Ordonnance n°2020-115 d

In [49]:
# Keep only changed articles in their content
AML5_modifs = AML5_modifs[AML5_modifs['Compare contenu'] =="OK change"]
AML5_modifs['Compare contenu'].unique()

array(['OK change'], dtype=object)

In [50]:
# Delete KO in Old and new articles
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Ancien_Article'].str.contains("KO", na=False)]
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Nouv_Vers_Article'].str.contains("KO", na=False)]

In [56]:
len(AML5_modifs)

54

In [53]:
display(AML5_modifs)

,Année,Version du,Date de début cible d'entrée en vigueur,Est la dernière version,ID Article Modificateur,Titre Article Modificateur,Nature Article Modificateur,Date de début cible Article Modificateur,Action Article Modificateur,ID Parent,Nom Parent,CID Parent,ID Article Cible,Titre Article Cible,Date de début (Article Cible),Date de fin (Article Cible),Contenu_Ancien_Article,Contenu_Nouv_Vers_Article,Compare contenu
688,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644802,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000038509568,Chapitre X : Prestataires de services sur acti...,LEGISCTA000038509568,LEGIARTI000042648495,L54-10-3,2020-12-11,2024-01-01,"Avant d'exercer leur activité, les prestataire...","Avant d'exercer leur activité, les prestataire...",OK change
689,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644804,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000038509568,Chapitre X : Prestataires de services sur acti...,LEGISCTA000038509568,LEGIARTI000042648498,L54-10-4,2020-12-11,2024-10-18,L'exercice de la profession de prestataire des...,L'exercice de la profession de prestataire des...,OK change
690,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644806,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000038509568,Chapitre X : Prestataires de services sur acti...,LEGISCTA000038509568,LEGIARTI000042648501,L54-10-5,2020-12-11,2024-01-01,I.-Pour la fourniture à titre de profession ha...,I.-Pour la fourniture à titre de profession ha...,OK change
691,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644808,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000020196669,Section 3 : Obligations de vigilance à l'égard...,LEGISCTA000020179278,LEGIARTI000042648537,L561-14,2020-12-11,2999-01-01,Les personnes mentionnées aux 1° à 7° de l'art...,Les personnes mentionnées aux 1° à 7° bis de l...,OK change
694,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644808,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000033517859,Section 4 : Obligation de déclaration et d'inf...,LEGISCTA000020180500,LEGIARTI000042648504,L561-20,2020-12-11,2999-01-01,I. – Par dérogation à l'article L. 561-18 et s...,I. – Par dérogation à l'article L. 561-18 et s...,OK change
697,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644814,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000006170764,Section 4 : Autres prestataires de services,LEGISCTA000006170764,LEGIARTI000042648661,L745-11-9,2020-12-11,2022-02-26,"I. - Sont applicables en Nouvelle-Calédonie, s...","I.-Sont applicables en Nouvelle-Calédonie, sou...",OK change
698,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644816,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000006170893,Section 6 : Obligations relatives à la lutte c...,LEGISCTA000006170893,LEGIARTI000042653399,L745-13,2020-12-11,2021-07-22,I. – Outre les dispositions du chapitre Ier du...,I. – Outre les dispositions du chapitre Ier du...,OK change
699,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644816,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000006170898,Section 6 : Obligations relatives à la lutte c...,LEGISCTA000006170898,LEGIARTI000042653352,L755-13,2020-12-11,2021-07-22,I. – Outre les dispositions du chapitre Ier du...,I. – Outre les dispositions du chapitre Ier du...,OK change
700,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644818,Ordonnance n°2020-1544 du 9 décembre 2020 - ar...,CODE,2020-12-11,MODIFICATION,LEGISCTA000006170817,Section 4 : Autres prestataires de services,LEGISCTA000006170817,LEGIARTI000042648666,L765-11-9,2020-12-11,2022-02-26,I. - Sont applicables dans les îles Wallis et ...,I.-Sont applicables dans les îles Wallis et Fu...,OK change
701,2020,2020-12-11,2020-12-11,False,LEGIARTI000042644820,Ordonna